In [10]:
import os

import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from datetime import datetime, timedelta
from torch.utils.data import DataLoader, random_split, TensorDataset
from torchvision.datasets import MNIST

device = "cuda" if torch.cuda.is_available() else "cpu"

In [24]:
class Generator(nn.Module):
    def __init__(self, product_dims, customer_dims):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(product_dims, 64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(64, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(64, 32),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(32, customer_dims)
        )

    def forward(self, product_vector):
        return self.model(product_vector)

In [25]:
class Discriminator(nn.Module):
    def __init__(self, customer_dims):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(customer_dims, 64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(64, 64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(64, 32),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(32, 32),
            nn.Linear(32, 1),
            nn.Sigmoid(),
        )

    def forward(self, customer_vector):
        return self.model(customer_vector)

In [13]:
# Products data
products = pd.read_csv('data/products.csv')

prods_final = pd.concat([
    pd.get_dummies(products['CATEGORY'])
], axis=1)
prods_final['price'] = products['PRICE']/products['PRICE'].max()
prods_final['PRODUCT_ID'] = products['ID'].copy()

In [14]:
# Customer data
customers = pd.read_csv('data/customers.csv')

customers_final = pd.concat([
    pd.get_dummies(customers['GENDER'])[['Female', 'Male']],
    pd.get_dummies(customers['COUNTRY']),
    pd.get_dummies(customers['TYPE'])
], axis=1)
customers_final['age'] = ((datetime.today() - pd.to_datetime(customers['DOB']))/timedelta(days=365)).round()/100
customers_final['CUSTOMER_ID'] = customers['ID'].copy()

In [15]:
# Training data
transactions = pd.read_csv('data/trans_small_subset.csv')[['PRODUCT_ID', 'CUSTOMER_ID']].iloc[0:500000]
X = transactions.merge(prods_final, how='left', on='PRODUCT_ID').drop(columns=['PRODUCT_ID', 'CUSTOMER_ID'])
y = transactions.merge(customers_final, how='left', on='CUSTOMER_ID').drop(columns=['PRODUCT_ID', 'CUSTOMER_ID'])

In [16]:
X_nans_mask = X.isna().all(axis=1)
X = X[~X_nans_mask]
y = y[~X_nans_mask]

y_nans_mask = y.isna().all(axis=1)
X = X[~y_nans_mask].reset_index(drop=True)
y = y[~y_nans_mask].reset_index(drop=True)

In [35]:
train_size = 0.8
batch_size = 1024
lr = 3e-4
num_epochs = 50
product_dims = len(X.columns)
customer_dims = len(y.columns)
disc_input_dims = product_dims + customer_dims
fixed_noise = torch.randn((batch_size, product_dims)).to(device)
step = 0

disc = Discriminator(disc_input_dims).to(device)
gen = Generator(product_dims, customer_dims).to(device)
criterion = nn.BCELoss()
opt_disc = optim.Adam(disc.parameters(), lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)

train_inds = random.sample(list(X.index.values), int(train_size*len(X.index)))
input_data = torch.tensor(X.iloc[train_inds].values.astype(np.float32))
true_positive = torch.tensor(y.iloc[train_inds].values.astype(np.float32))
labels = torch.ones_like(input_data)
train_dataset = TensorDataset(
    input_data,
    true_positive
)
loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)

In [41]:
torch.set_printoptions(precision=3, sci_mode=False, profile='default')

n_total_steps = len(loader)
for epoch in range(num_epochs):
    for batch_idx, (input_prod, true_positive) in enumerate(loader):
        input_prod = input_prod.to(device)
        batch_size = input_prod.shape[0]
        
        # Train Discriminator: max log(D(real)) + log(1 - D(G(z)))
        #noise = torch.randn(batch_size, product_dims)
        disc_input_tp = torch.cat((true_positive, input_prod), 1)
        disc_tp = disc(disc_input_tp).view(-1)
        lossD_tp = criterion(disc_tp, torch.ones_like(disc_tp))
        
        fake = gen(input_prod)
        disc_input_fake = torch.cat((fake, input_prod), 1)
        disc_fake = disc(disc_input_fake).view(-1)
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        lossD = (lossD_tp + lossD_fake)/2
        disc.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()
        
        # Train Generator min log(1 - D(G(z))) -> max log(D(G(z)))
        output = disc(disc_input_fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()
        
        if batch_idx == 0:# or ((batch_idx + 1) % 50 == 0):
            print(
                f"Epoch [{epoch}/{num_epochs}] \ "
                f"Step [{batch_idx + 1}/{n_total_steps}] \ "
                f"Loss D: {lossD:.3f}, Loss G: {lossG:.3f}"
            )
            
            if epoch % 5 == 0:
                with torch.no_grad():
                    fake = gen(fixed_noise)
                    print(fake)

Epoch [0/50] \ Step [1/336] \ Loss D: 0.694, Loss G: 0.771
tensor([[-0.029,  0.087, -0.028,  ...,  0.178,  0.123,  0.004],
        [-0.034,  0.085, -0.027,  ...,  0.176,  0.123,  0.002],
        [-0.036,  0.087, -0.029,  ...,  0.179,  0.120,  0.002],
        ...,
        [-0.035,  0.082, -0.033,  ...,  0.178,  0.121,  0.004],
        [-0.037,  0.087, -0.028,  ...,  0.181,  0.121,  0.001],
        [-0.036,  0.087, -0.029,  ...,  0.180,  0.121,  0.000]])
Epoch [1/50] \ Step [1/336] \ Loss D: 0.696, Loss G: 0.699
Epoch [2/50] \ Step [1/336] \ Loss D: 0.704, Loss G: 0.709
Epoch [3/50] \ Step [1/336] \ Loss D: 0.720, Loss G: 0.696
Epoch [4/50] \ Step [1/336] \ Loss D: 0.697, Loss G: 0.686
Epoch [5/50] \ Step [1/336] \ Loss D: 0.689, Loss G: 0.698
tensor([[    -0.716,      4.855,     -3.047,  ...,     -3.284,      4.115,
             -0.308],
        [    -0.630,      4.218,     -2.567,  ...,     -2.777,      3.542,
             -0.218],
        [    -0.424,      2.932,     -1.623,  ...,    

In [ ]:
"""
test_dataset = TensorDataset(
    torch.tensor(X[~X.index.isin(train_inds)].values.astype(np.float32)),
    torch.tensor(y[~y.index.isin(train_inds)].values.astype(np.float32))
)
"""


# test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)

In [ ]:
customers

In [ ]:
products = pd.read_csv('data/products.csv')
products.head()

In [ ]:
pd.get_dummies(products['CATEGORY']).sum()

In [ ]:
X = pd.concat([
    pd.get_dummies(products['CATEGORY'])
], axis=1)
X['price'] = products['PRICE']/products['PRICE'].max()

In [ ]:
X

In [ ]:
transactions